In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/BTP

/content/drive/MyDrive/BTP


In [3]:
import argparse
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
#import dataset
from datetime import datetime
import random
import numpy as np
import copy
import torchvision.models as models
#import detector_net
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import os.path
import lid
import activations_extraction

In [4]:
import os
import math
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import torch.optim as optim

In [5]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
batch_size = 64
epochs = 10
log_interval = 100
test_interval = 1
p = 100
stage = 1

In [7]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.3681, 0.3813, 0.3448],[0.1505, 0.1406, 0.1377])
])

In [8]:
train_set=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/BTP/dataset/train', transform=transformer)
train_loader=DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [9]:
out1 = 8
out2 = 16
out3 = 32

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import time
import numpy as np
import matplotlib.pyplot as plt

size_1 = 64
size_2 = 64

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 3 input channels for CIFAR10, VGG11 calls for 64 output channels from
        # the first conv layer, a batchnorm, then a ReLU
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1,bias=True)# 3 8
        # self.norm1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU()

        self.mp1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1,bias=True)# 8 16
        # self.norm2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        self.mp2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1,bias=True)# 16 32
        # self.norm3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1,bias=True)# 32 32
        # self.norm4 = nn.BatchNorm2d(32)
        self.relu4 = nn.ReLU()
        #
        self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1,bias=True)# 32 64
        # self.norm5 = nn.BatchNorm2d(64)
        self.relu5 = nn.ReLU()
        #
        # self.conv6 = nn.Conv2d(size_1, 64, kernel_size=3, padding=1)
        # self.norm6 = nn.BatchNorm2d(64)
        # self.relu6 = nn.ReLU()

    def forward(self, x0, stage):
        # print(x0.size())
        # self.conv1.weight = torch.nn.Parameter(self.conv1.weight.abs())
        x1 = self.conv1(x0)
        # feature = x1
        # x1,_ = BinActive2.apply(x1, 255)
        if stage == 0:

            soi = x1.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x1 = self.norm1(x1)
        # x1,_ = BinActive2.apply(x1, 255)
        x1 = self.relu1(x1)
        # x1,_ = BinActive2.apply(x1, 255)
        # x1 = self.mp1(x1)

        x2 = self.conv2(x1)
        # x2,_ = BinActive2.apply(x2, 255)
        if stage == 1:
            soi = x2.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x2 = self.norm2(x2)
        # x2,_ = BinActive2.apply(x2, 255)

        # soi = x2.abs().mean(dim=1).mean(dim=1).mean(dim=1)

        x2 = self.relu2(x2)
        # x2,_ = BinActive2.apply(x2, 255)

        # x2 = self.mp2(x2)
        x3 = self.conv3(x2)
        # x3,_ = BinActive2.apply(x3, 255)
        if stage == 2:
            soi = x3.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x3 = self.norm3(x3)
        # x3,_ = BinActive2.apply(x3, 255)

        # soi = x3.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        x3 = self.relu3(x3)
        # x3,_ = BinActive2.apply(x3, 255)

        # x3 = self.mp2(x3)
        x4 = self.conv4(x3)
        # x4,_ = BinActive2.apply(x4, 255)

        # soi = x4.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x4 = self.norm4(x4)
        # x4,_ = BinActive2.apply(x4, 255)

        x4 = self.relu4(x4)
        # x4,_ = BinActive2.apply(x4, 255)

        #
        x5 = self.conv5(x4)
        # x5,_ = BinActive2.apply(x5, 255)

        # soi = x5.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x5 = self.norm5(x5)
        # x5,_ = BinActive2.apply(x5, 255)

        # x5 = self.relu5(x5)
        # x5,_ = BinActive2.apply(x5, 255)

        #
        # x6 = self.conv6(x5)
        # # soi = x6.abs().mean(dim=1).mean(dim=1).mean(dim=1)
        # x6 = self.norm6(x6)
        # x6 = self.relu6(x6)

        return soi


In [11]:
model = Net()
model.conv1 = nn.Conv2d(3, out1, kernel_size=3, padding=1,bias=True)
model.conv2 = nn.Conv2d(out1, out2, kernel_size=3, padding=1, bias=True)
model.conv3 = nn.Conv2d(out2, out3, kernel_size=3, padding=1, bias=True)
model.conv4 = nn.Conv2d(out3, 32, kernel_size=3, padding=1, bias=True)
print(model)

Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (conv5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5): ReLU()
)


In [12]:
la = 0.9
la_list = [la, la+0.4,la+1.4, 2.5, 3.0]
lc = 0.1
lr_list = [0.002, 0.002, 0.0005, 2.5, 3.0]

la = la_list[stage]
lr = lr_list[stage]

In [13]:
model = torch.nn.DataParallel(model)

In [14]:
if stage > 0: model.load_state_dict(torch.load('/content/drive/MyDrive/BTP/les_training.pth'))

In [15]:
model = model.cuda()
optimizer = optim.SGD(model.parameters(), lr= lr, weight_decay=5e-4, momentum=0.6)

In [16]:
if stage >= 1:
    model.module.conv1.weight.requires_grad = False
    model.module.conv1.bias.requires_grad = False
if stage >= 2:
    model.module.conv2.weight.requires_grad = False
    model.module.conv2.bias.requires_grad = False

if stage >= 3:
    model.module.conv3.weight.requires_grad = False
    model.module.conv3.bias.requires_grad = False

if stage >= 4:
    model.module.conv4.weight.requires_grad = False
    model.module.conv4.bias.requires_grad = False

In [17]:
best_acc, old_file = 0, None
best_loss = 1000
best_sep = 0
lambda_adv = torch.tensor([la]).cuda()
lambda_clean = torch.tensor([lc]).cuda()
t_begin = time.time()
mseloss = torch.nn.MSELoss()
celoss = torch.nn.CrossEntropyLoss()
n_batches = int(250 * p * 0.01)

In [ ]:
for epoch in range(epochs):
    print(f'epoch : {epoch}')
    model.train()
    
    print("training phase")

    for batch_idx, (data, y) in enumerate(train_loader):
        print('batch idx: ', batch_idx)
        
        if batch_idx < n_batches:
            #indx_target = target.clone()

            activations = activations_extraction.extract_activations(data)  
            print("activations: ", activations) 
            features = lid.compute_lid(activations)
            print("LID features: ", features)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ;features = data.clone()

            features, y = features.cuda(), y.cuda()
            features, y  = Variable(features), Variable(y)
            optimizer.zero_grad()

            
  
            # output = model(data)
            # x = data
            # for i in range(8):
            #     x = model.module.features[i](x)
            sum_of_I = model(features, stage)
            
            # sum_of_I = x.abs().mean(dim=1).mean(dim=1).mean(dim=1)
            # sum_of_I = model.module.features[3](data).abs().mean(dim=1).mean(dim=1).mean(dim=1)
            # print(sum_of_I.size())
            loss =  ((y)*mseloss(sum_of_I,lambda_adv))+((1-y)*mseloss(sum_of_I,lambda_clean))
            loss.sum().backward()

            optimizer.step()
            
        # if batch_idx % args.log_interval == 0 and batch_idx > 0:

            # print(f'adv_energy_loss : {(y*mseloss(sum_of_I,lambda_adv)).sum()} clean_energy_loss : {((1-y)*mseloss(sum_of_I,lambda_clean)).sum()}')
    
    print(f'adv_energy_loss : {(y*mseloss(sum_of_I,lambda_adv)).sum()} clean_energy_loss : {((1-y)*mseloss(sum_of_I,lambda_clean)).sum()}')
    
    elapse_time = time.time() - t_begin
    speed_epoch = elapse_time / (epoch + 1)
    speed_batch = speed_epoch / 50000
    eta = speed_epoch * epochs - elapse_time
    torch.save(model.state_dict(), '/content/drive/MyDrive/BTP/les_training.pth')

print("total time: ", elapse_time, "epoch speed: ", speed_epoch,'batch speed: ',speed_batch, 'eta: ',eta)
    